# Customer Segmentation - 고객 군집

In [8]:
import warnings
warnings.filterwarnings('ignore')

# I. Import Packages and Google Drice Mount

> ## 1) Import Packages

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

> ## 2) Google Drive Mount

- 'Online_Retail.zip' 파일을 구글드라이브에 업로드 후 진행

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls -l '/content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip'

-rw------- 1 root root 22824989 Mar 11 00:25 '/content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip'


# II. Data Preprocessing

> ## 1) Unzip 'Online_Retail.zip'

- Colab 파일 시스템에 'Online_Retail.csv' 파일 생성

In [5]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/datasets/Online_Retail.zip

Archive:  /content/drive/My Drive/Colab Notebooks/datasets/Online_Retail.zip
  inflating: Online_Retail.xlsx      


- Online_Retail.zip 파일 확인

In [6]:
!ls -l

total 23168
drwx------ 5 root root     4096 Mar 11 00:28 drive
-rw-r--r-- 1 root root 23715344 Mar  7 16:08 Online_Retail.xlsx
drwxr-xr-x 1 root root     4096 Mar  5 14:37 sample_data


> ## 2) 데이터 읽어오기

- pandas DataFrame

In [106]:
%%time

DF = pd.read_excel('Online_Retail.xlsx')

DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB
CPU times: user 39.6 s, sys: 60.7 ms, total: 39.7 s
Wall time: 39.7 s


> ## 3) 데이터 설명

- InvoiceNo : 주문번호, 'C' 시작은 주문 취소
- StockCode : 제품 코드(Item Code)
- Description : 제품 설명
- Quantity : 주문 건수
- InvoiceDate : 주문 날짜
- UnitPrice : 제품 단가
- CustomerID : 고객번호
- Country : 국가명(주문 고객 국적)

In [11]:
DF.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


> ## 4) 결측치 제거

- 'Quantity', 'UnitPrice', 'CustomerID'

In [107]:
DF = DF[DF['Quantity'] > 0]
DF = DF[DF['UnitPrice'] > 0]
DF = DF[DF['CustomerID'].notnull()]

DF.shape

(397884, 8)

- 결과 확인

In [108]:
DF.isnull().sum(axis = 0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

> ## 5) 'United Kingdom'만 사용

- 대부분의 구매자가 영국 국적

In [109]:
DF['Country'].value_counts()[:10]

United Kingdom    354321
Germany             9040
France              8341
EIRE                7236
Spain               2484
Netherlands         2359
Belgium             2031
Switzerland         1841
Portugal            1462
Australia           1182
Name: Country, dtype: int64

- 영국 데이터만 추출

In [110]:
DF = DF[DF['Country'] == 'United Kingdom']

DF.shape

(354321, 8)

In [111]:
DF.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


> ## 6) 불필요한 Column 제거

- Country는 모두 동일
- Description은 StockCode로 구분 가능
- InvoiceDate는 InvoiceNo와 CustomerID로 구분 가능


In [112]:
DF.drop('Country', axis = 1, inplace = True)
DF.drop('InvoiceDate', axis = 1, inplace = True)
DF.drop('Description', axis = 1, inplace = True)

In [113]:
DF.head()

,InvoiceNo,StockCode,Quantity,UnitPrice,CustomerID
0,536365,85123A,6,2.55,17850.0
1,536365,71053,6,3.39,17850.0
2,536365,84406B,8,2.75,17850.0
3,536365,84029G,6,3.39,17850.0
4,536365,84029E,6,3.39,17850.0


- Quantity, UnitPrice는 totalPrice로 변환

In [114]:
DF['totalPrice'] = DF['Quantity'] * DF['UnitPrice']

In [115]:
DF.drop('Quantity', axis = 1, inplace = True)
DF.drop('UnitPrice', axis = 1, inplace = True)

In [116]:
DF.head()

,InvoiceNo,StockCode,CustomerID,totalPrice
0,536365,85123A,17850.0,15.30
1,536365,71053,17850.0,20.34
2,536365,84406B,17850.0,22.00
3,536365,84029G,17850.0,20.34
4,536365,84029E,17850.0,20.34


- CustomerID 정수 변환

In [117]:
DF['CustomerID'] = DF['CustomerID'].astype(int)

# III. EDA

- 고객당 주문 수

In [25]:
DF['CustomerID'].value_counts()

17841.0    7983
14911.0    5903
14096.0    5128
12748.0    4642
14606.0    2782
           ... 
13391.0       1
16579.0       1
13366.0       1
17763.0       1
17846.0       1
Name: CustomerID, Length: 4372, dtype: int64

- 주문당 물품 개수

In [43]:
DF['InvoiceNo'].value_counts()

573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
C537820       1
548205        1
548206        1
548207        1
C558897       1
Name: InvoiceNo, Length: 25900, dtype: int64

# IV. K-means

> ## 1) Standardization

In [118]:
X = DF[['InvoiceNo', 'CustomerID', 'totalPrice']]

In [119]:
from sklearn.preprocessing import StandardScaler

X_Scaled = StandardScaler().fit_transform(X)

> ## 2) Modeling

In [120]:
from sklearn.cluster import KMeans

kmeans_6 = KMeans(n_clusters = 6,
                  init = 'k-means++',
                  # max_iter = 20,
                  random_state = 2045)

kmeans_6.fit(X_Scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=2045, tol=0.0001, verbose=0)

> ## 2) 'Clustering' 추가

In [126]:
DF['Clustering_6'] = kmeans_6.labels_

> ## 3) 실루엣 점수

- silhoutte_score()

In [129]:
from sklearn.metrics import silhouette_score

silhouette_score(X_Scaled, DF['Clustering_6'])

0.4175190043658813